In [1]:
import json

with open("../api_name.json", "r") as f:
    api_names = json.load(f)

with open("../api_category.json", "r") as f:
    api_categories = json.load(f)

with open("../mashup_name.json", "r") as f:
    mashup_names = json.load(f)

with open("../mashup_category.json", "r") as f:
    mashup_categories = json.load(f)

with open("../mashup_used_api.json", "r") as f:
    mashup_used_apis = json.load(f)

In [2]:
warm_start_mashup_indices = []

for i, mashup_used_api in enumerate(mashup_used_apis):
    if len(mashup_used_api) >= 5:
        warm_start_mashup_indices.append(i)

warm_start_api_indices = []
warm_start_mashup_api_interactions = []
for mashup_i, warm_start_mashup_index in enumerate(warm_start_mashup_indices):
    interactions = []
    mashup_used_api = mashup_used_apis[warm_start_mashup_index]
    # print(mashup_name)
    for api_name in mashup_used_api:
        api_index = api_names.index(api_name)
        if api_index not in warm_start_api_indices:
            warm_start_api_indices.append(api_index)
        interactions.append(warm_start_api_indices.index(api_index))
    warm_start_mashup_api_interactions.append(interactions)

print("Warm start mashup indices:", len(warm_start_mashup_indices))
print("Warm start API indices:", len(warm_start_api_indices))

print("Warm start mashup API interactions:", warm_start_mashup_api_interactions[:5])

with open("train.txt", "w") as f:
    with open("test.txt", "w") as f_test:
        for i, interactions in enumerate(warm_start_mashup_api_interactions):
            train_size = int(len(interactions) * 0.8)
            train_interactions = interactions[:train_size]
            test_interactions = interactions[train_size:]
            train_string = " ".join([str(x) for x in train_interactions])
            test_string = " ".join([str(x) for x in test_interactions])
            train_string = str(i) + " " + train_string
            test_string = str(i) + " " + test_string
            f.write(train_string + "\n")
            f_test.write(test_string + "\n")
f_test.close()
f.close()

Warm start mashup indices: 607
Warm start API indices: 893
Warm start mashup API interactions: [[0, 1, 2, 3, 4], [5, 1, 6, 7, 8, 9], [10, 11, 1, 12, 13, 14, 15, 16], [15, 1, 17, 18, 19], [1, 18, 20, 21, 22, 23, 24, 25]]


In [3]:
# build co_category-api matrix
co_category_api_matrix = []
for i, warm_start_api_index in enumerate(warm_start_api_indices):
    i_categories = api_categories[warm_start_api_index]
    for j, warm_start_api_index in enumerate(warm_start_api_indices):
        if i == j:
            continue
        j_categories = api_categories[warm_start_api_index]
        common_categories = set(i_categories) & set(j_categories)
        if len(common_categories) > 0:
            co_category_api_matrix.append((i, j, len(common_categories)))

with open("api_co_category.txt", "w") as f:
    for i, j, count in co_category_api_matrix:
        f.write(f"{i} {j} {count}\n")

In [4]:
co_category_mashup_matrix = []
for i, warm_start_mashup_index in enumerate(warm_start_mashup_indices):
    i_categories = mashup_categories[warm_start_mashup_index]
    for j, warm_start_mashup_index in enumerate(warm_start_mashup_indices):
        if i == j:
            continue
        j_categories = mashup_categories[warm_start_mashup_index]
        common_categories = set(i_categories) & set(j_categories)
        if len(common_categories) > 0:
            co_category_mashup_matrix.append((i, j, len(common_categories)))

with open("mashup_co_category.txt", "w") as f:
    for i, j, count in co_category_mashup_matrix:
        f.write(f"{i} {j} {count}\n")

In [5]:
print("Co-category API matrix:", len(co_category_api_matrix))
print("Co-category Mashup matrix:", len(co_category_mashup_matrix))

Co-category API matrix: 51308
Co-category Mashup matrix: 150402


In [6]:
co_api_mashup_matrix = []
for i, warm_start_mashup_api_interaction in enumerate(warm_start_mashup_api_interactions):
    i_interactions = warm_start_mashup_api_interaction
    for j, warm_start_mashup_api_interaction in enumerate(warm_start_mashup_api_interactions):
        if i == j:
            continue
        j_interactions = warm_start_mashup_api_interaction
        common_interactions = set(i_interactions) & set(j_interactions)
        if len(common_interactions) > 0:
            co_api_mashup_matrix.append((i, j, len(common_interactions)))

with open("mashup_co_api.txt", "w") as f:
    for i, j, count in co_api_mashup_matrix:
        f.write(f"{i} {j} {count}\n")

In [7]:
print("Co-api mashup matrix:", len(co_api_mashup_matrix))

Co-api mashup matrix: 174506


In [ ]:
import numpy as np

co_mashup_api_matrix = np.zeros((len(warm_start_api_indices), len(warm_start_api_indices)))
for interactions in warm_start_mashup_api_interactions:
    for i in interactions:
        for j in interactions:
            if i != j:
                co_mashup_api_matrix[i][j] += 1

# save as numpy file
np.save("co_mashup_api_matrix.npy", co_mashup_api_matrix)
# save as csr matrix
from scipy.sparse import csr_matrix, save_npz
co_mashup_api_csr = csr_matrix(co_mashup_api_matrix)
with open("co_mashup_api_matrix.npz", "wb") as f:
    save_npz(f, co_mashup_api_csr)

4 5
9 6
16 8
19 5
25 8
28 5
31 5
34 5
34 5
37 5
41 8
45 5
49 5
52 8
59 14
62 6
69 12
86 21
90 6
93 6
96 9
99 6
103 6
54 6
105 6
108 8
112 9
117 9
119 9
121 6
125 8
129 9
132 11
54 5
133 5
134 5
135 9
138 10
139 6
141 6
121 6
143 7
56 6
130 6
146 5
147 5
148 5
54 5
151 6
153 11
159 19
160 12
174 29
140 5
178 9
179 6
180 9
182 6
127 5
184 8
128 6
185 5
143 9
186 10
187 7
189 6
193 8
131 6
192 9
195 5
196 9
48 5
199 9
57 5
203 15
205 6
40 5
208 8
40 5
209 6
210 5
199 20
160 5
212 7
54 7
214 6
42 5
100 5
215 6
189 14
217 13
179 9
222 9
223 7
229 9
48 6
234 15
211 10
235 10
238 13
239 7
240 10
218 5
241 5
242 9
188 5
245 16
246 6
190 8
136 8
248 5
54 7
131 8
249 5
215 5
250 5
254 11
255 5
258 18
261 21
262 7
179 5
265 7
266 6
267 5
163 6
269 6
218 8
175 6
249 6
271 5
272 5
273 5
230 5
275 8
277 14
278 7
136 5
279 6
243 8
282 11
284 5
140 6
155 10
54 5
285 6
288 5
290 14
249 5
294 12
284 7
274 5
259 9
289 5
295 5
298 9
301 8
169 6
304 8
307 13
309 5
311 5
315 8
315 8
236 6
317 8
321 7
56 5
3